---
# Trabalho 1 - Tópicos Especiais em Matemática Aplicada

**Alunos/Matricula:** João V. Farias & Renan V. Guedes / 221022604 & 221031363

**Arquitetura Usada:** Encoder-Decoder

**Dataset Link:** V1: [D-Talk](https://www.tensorflow.org/datasets/catalog/ted_hrlr_translate#ted_hrlr_translatefr_to_pt) from TensorFlow.Datasets

---

### **Projeto para traduzir mensagens do Francês para o Portugues**  

Neste projeto, vamos explorar e comparar três arquiteturas de redes neurais para tradução automática do francês para o português, usando o *dataset* TED Talks do *Open Translation Project*. A ideia é testar modelos do tipo **Encoder-Decoder**, analisando suas diferenças e impacto na qualidade da tradução.  

Os três modelos que vamos treinar são:  

1. **LSTM (Long Short-Term Memory)**  
   - Um modelo básico de rede recorrente bidirecional. O **Encoder** processa a frase em francês e gera um contexto, enquanto o **Decoder** usa esse contexto para formar a tradução em português.  
   - A principal vantagem desse modelo é sua capacidade de lidar com dependências de longo prazo nas sequências.  

2. **LSTM com Mecanismos de Atenção**  
   - Uma versão aprimorada do modelo anterior, adicionando camadas de atenção (produto escalar, Bahdanau e Luong).  
   - A atenção ajuda o modelo a "olhar" para partes específicas da frase de entrada enquanto traduz, melhorando a coerência e precisão.  

3. **Transformers**  
   - Uma abordagem mais moderna, baseada em **autoatenção**, eliminando o uso de redes recorrentes.  
   - Trabalha com processamento paralelo, usando *Multi-Head Attention* e *Positional Encoding* para entender relações entre palavras, mesmo quando estão distantes na frase.  

**Como vamos testar os modelos?**  
- **Dataset**: Vamos usar cerca de 52.000 pares de frases (francês-português) para treinar, além de 1.200 para validação e 1.800 para teste.  
- **Pré-processamento**: Faremos a tokenização com *SubwordTextEncoder* para reduzir palavras fora do vocabulário (*out-of-vocabulary* – OOV).  
- **Treinamento**: Otimização com Adam, acompanhando a perda (*loss*) e a acurácia durante o processo.  
- **Avaliação**: Vamos comparar os resultados usando a métrica BLEU e analisar exemplos práticos das traduções.  

**O que esperamos encontrar?**  
- Nosso objetivo é entender qual desses modelos tem o melhor equilíbrio entre qualidade de tradução e eficiência computacional.  
- É provável que os Transformers tenham um desempenho superior, já que conseguem processar frases de forma mais eficiente, enquanto os modelos com LSTM e atenção devem mostrar um avanço significativo sobre a versão básica de LSTM.  

No fim das contas, essa análise pode ajudar a compreender melhor como diferentes abordagens de deep learning se saem em tarefas de tradução, trazendo insights úteis para aplicações reais em NLP.

---

## 📚 Importando as bibliotecas necessárias

In [2]:
# Primeiro, vamos importar todas as bibliotecas que vamos precisar ao longo do Projeto
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import time
import os

# Para processamento de texto
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Componentes do Keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention, MultiHeadAttention
from tensorflow.keras.optimizers import Adam

tf.config.optimizer.set_jit(True)  # Ativa o XLA JIT compilation
tf.keras.mixed_precision.set_global_policy('mixed_float16') # Força o TensorFlow a usar precisão mista

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


## 🔍 Carregando e Preparando os Dados

In [3]:
#Agora vamos carregar nosso dataset de tradução
examples, metadata = tfds.load('ted_hrlr_translate/fr_to_pt', with_info=True,
                             as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

# Função para preprocessar o texto
def preprocess_text(text):
    # Adiciona espaços entre pontuação
    text = tf.strings.regex_replace(text, r"([?.!,¿])", r" \1 ")
    # Remove caracteres especiais
    text = tf.strings.regex_replace(text, r'[" "]+', " ")
    # Remove espaços extras
    text = tf.strings.strip(text)
    # Adiciona tokens de início e fim
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

# Vamos processar nossos dados
def prepare_dataset(pt, fr):
    pt = preprocess_text(pt)
    fr = preprocess_text(fr)
    return pt, fr

# Aplicando o preprocessamento
train_dataset = train_examples.map(prepare_dataset)
val_dataset = val_examples.map(prepare_dataset)

print("\n Dataset carregado e preprocessado com sucesso! 🎉")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/43873 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1131 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/incomplete.1PYMA9_1.0.0/ted_hrlr_translate-val…

Generating test examples...:   0%|          | 0/1494 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/incomplete.1PYMA9_1.0.0/ted_hrlr_translate-tes…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/1.0.0. Subsequent calls will reuse this data.

 Dataset carregado e preprocessado com sucesso! 🎉


## 🛠️ Configurando os Tokenizers

In [4]:
# 🛠️ Configurando os Tokenizers (versão otimizada)
MAX_SAMPLES = 50000  # Limite de amostras para processamento mais rápido

# Coletando os textos para treinamento dos tokenizers
pt_texts = []
fr_texts = []
for pt, fr in train_dataset.take(MAX_SAMPLES):
    pt_texts.append(pt.numpy().decode('utf-8'))
    fr_texts.append(fr.numpy().decode('utf-8'))

# Criando e treinando os tokenizers de uma vez
tokenizer_pt = Tokenizer(filters='')
tokenizer_fr = Tokenizer(filters='')

# Treinamento em batch
tokenizer_pt.fit_on_texts(pt_texts)
tokenizer_fr.fit_on_texts(fr_texts)

print("Vocabulário em Português:", len(tokenizer_pt.word_index))
print("Vocabulário em Francês:", len(tokenizer_fr.word_index))

Vocabulário em Português: 36969
Vocabulário em Francês: 37511
